Generates a CDM json-metadata file for the CRM entities handled by mapping and modelling processes.

In [ ]:
adls_root = 'https://stp1prodbra.blob.core.windows.net/commondataservice-sondamigraci-org187dcebc/Microsoft.Athena.TrickleFeedService'
cdm_entities = ['opportunity','account','businessunit','product','opportunityproduct','systemuser','cmtx_aprobacion','cmtx_costooportunidad',
                'cmtx_costoproducto','cmtx_filial','cmtx_pais','cmtx_grupo_divisional','cmtx_unidad','cmtx_usuariounidad']

In [2]:
results=[]

In [3]:
import requests
import json
import time
import pandas as pd

In [4]:
for cdm_entity in cdm_entities:
        
    cdm_config = {
        'cdm-entity':cdm_entity,
        'cdm-manifest-url':f'{adls_root}/{cdm_entity}-EntityMetadata.json',
        'cdm-metadata-key':[{'key':'StateMetadata','cols':['statecode','State','LocalizedLabelLanguageCode','LocalizedLabel'],'class':'option'},
                            {'key':'StatusMetadata','cols':['statuscode','Status','LocalizedLabelLanguageCode','LocalizedLabel'],'class':'option'},
                            {'key':'GlobalOptionSetMetadata','cols':['OptionSetName','Option','LocalizedLabelLanguageCode','LocalizedLabel'],'class':'option'},
                            {'key':'OptionSetMetadata','cols':['OptionSetName','Option','LocalizedLabelLanguageCode','LocalizedLabel'],'class':'option'}]
                            
    }

    jfile = requests.get(cdm_config['cdm-manifest-url'])
    data = jfile.json() 

    for cdm_session in cdm_config['cdm-metadata-key']:

        for x in data[cdm_session['key']]:

            collected_data = {
            'cdm_entity':cdm_config['cdm-entity'],
            'cdm_session':cdm_session['key'],    
            'cdm_object':cdm_session['cols'][0] if 'Option' not in cdm_session['key'] else x[cdm_session['cols'][0]],
            'cdm_value':x[cdm_session['cols'][1]],
            'cdm_lang':x[cdm_session['cols'][2]],
            'cdm_description':x[cdm_session['cols'][3]]
            }
            results.append(collected_data)

In [5]:

df = pd.DataFrame(results)
pd.options.display.float_format = '{:.0f}'.format


In [6]:

print(df[['cdm_entity','cdm_object','cdm_value','cdm_lang','cdm_description']][(df['cdm_entity']=='opportunity') & (df['cdm_session']=='StatusMetadata')])

     cdm_entity  cdm_object  cdm_value  cdm_lang        cdm_description
6   opportunity  statuscode          1      3082               En curso
7   opportunity  statuscode          1      1046           Em andamento
8   opportunity  statuscode  800000001      3082   Pendiente aprobación
9   opportunity  statuscode  800000001      1046     Aprovação pendente
10  opportunity  statuscode          3      3082                Lograda
11  opportunity  statuscode          3      1046                  Ganha
12  opportunity  statuscode          4      3082              Cancelado
13  opportunity  statuscode          4      1046              Cancelada
14  opportunity  statuscode          5      3082             Abandonado
15  opportunity  statuscode          5      1046               Esgotada
16  opportunity  statuscode  800000004      3082                Perdido
17  opportunity  statuscode  800000004      1046                Perdida
18  opportunity  statuscode  800000007      3082  Inactividad Co

In [8]:
data = json.dumps(results, indent=1) 

In [10]:
from azure.storage.blob import BlobServiceClient
from azure.storage.blob import BlobClient

In [11]:
connection_string = "DefaultEndpointsProtocol=https;AccountName=stp1prodeus;AccountKey=hUGd6VonByLafRA+PWI0PLFbvmXTEkvQYp4CAhJkGAR2PZdHIdNEYn7NNoo5j1u8V6KaKqGdcmTp/yq062MmYg==;EndpointSuffix=core.windows.net"
service = BlobServiceClient.from_connection_string(conn_str=connection_string)

In [12]:
blob = BlobClient.from_connection_string(conn_str=connection_string, container_name="acr-p1-prod-eus-003", blob_name="acr-p1-crm/sonda-crm-metadata.json")
blob.upload_blob(data, overwrite=True)

{'etag': '"0x8D8E7C939E4A6EF"',
 'last_modified': datetime.datetime(2021, 3, 15, 15, 44, 31, tzinfo=datetime.timezone.utc),
 'content_md5': bytearray(b'\x1a|\xf8\xb2}\xeb\x80\xaf\xcb\xf2\xa4\x18\xb5"\x9f\xb6'),
 'client_request_id': '5434a517-85a5-11eb-addd-a0afbd8edb8f',
 'request_id': 'adba0504-001e-0009-75b2-199abc000000',
 'version': '2020-06-12',
 'version_id': None,
 'date': datetime.datetime(2021, 3, 15, 15, 44, 31, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None}